# chunking

In [ ]:
from typing import List

def chunk(text: str, size=50, overlap=10):
    out, i = [], 0
    while i < len(text):
        out.append(text[i:i+size])  #text[0,50]->out
        i += max(1, size - overlap) # i=0 -> max(1, 50-40) = 40
    return out
    
chunked_list = chunk("A"*120, size=50, overlap=10)
print(len(chunked_list[2]))


# inverted index
This is the foundational structure behind search engines, keyword recall, RAG systems, and hybrid retrieval.

In [ ]:
from collections import defaultdict

docs = {
    1: "refund policy and receipt",
    2: "how to request a refund",
    3: "lost receipt replacement"
}

index = defaultdict(set)
print(list(index))  #[]

for doc_id, text in docs.items():   #kv pairs
    for word in text.lower().split():
        index[word].add(doc_id)

print(index["refund"])


[]
{1, 2}


In [1]:
import time
print("hello")
time.sleep(0.2)
print("done")


hello
done
